In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import fastf1
from src.plotset import setup_plot
from fastf1 import plotting

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
team_names = ['McLaren', 'Ferrari', 'Mercedes', 'Red Bull Racing', 'Williams',
              'Kick Sauber', 'Racing Bulls', 'Aston Martin', 'Haas F1 Team', 'Alpine']

In [ ]:
def mean_pit_time(session,team_names=team_names):
    pit_times = []
    for team in team_names:
        df = pd.DataFrame(session.laps.pick_teams(team))
        df['PitTotalTime'] = df.PitOutTime.shift(-1) - df.PitInTime
        df['PitTotalTime'] = df['PitTotalTime'].dt.total_seconds()
        pit_times.append(round(df.PitTotalTime.mean(),3))
    return pit_times

In [ ]:
pit_perf = {}
for i in range(1,13):
    race = fastf1.get_session(2025,i,'R')
    race.load(laps=True,telemetry=False,weather=False,messages=False)
    if i == 12:
        pit_perf[i] = np.float64(28.991)
    else:
        pit_perf[i] = mean_pit_time(race)

In [ ]:
pits_df = pd.DataFrame(pit_perf).T
team_names = ['McLaren', 'Ferrari', 'Mercedes', 'RedBull\nRacing', 'Williams',
              'Kick\nSauber', 'Racing\nBulls', 'Aston\nMartin', 'Haas\nF1 Team', 'Alpine']
pits_df.columns = team_names

In [ ]:
rel_pits_df = round(pits_df.div(pits_df.min(axis=1),axis=0),3)

In [ ]:
driver_names = ['NOR','LEC','RUS','VER','ALB',
                'HUL','LAW','ALO','OCO','GAS']

In [ ]:
driver_palette = plotting.get_driver_color_mapping(session=race)
team_palette = {}
for i,j in zip(team_names,driver_names):
    team_palette[i] = driver_palette[j]

In [ ]:
rel_pits_df.columns = team_names

In [ ]:
scores_df = 0.8*rel_pits_df.median() + 0.2*(rel_pits_df.quantile(0.75) - rel_pits_df.quantile(0.25))
scores_df.sort_values(inplace=True)

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))

sns.boxplot(data=rel_pits_df,orient='h',width=0.8,linecolor='w',linewidth=1,fliersize=0,
            order=scores_df.index,palette=team_palette,ax=ax,boxprops=dict(alpha=0.2))
sns.stripplot(data=rel_pits_df,orient='h',linewidth=1,size=10,edgecolor='#666666',
              order=scores_df.index,palette=team_palette,ax=ax)

ax.set_xlim(left=0.99,right=1.35)
ax.set_xlabel('Gap (%)')
ax.set_xticks([i/100 for i in range(100,140,5)])
ax.set_xticklabels(['Least\nTime Spent']+[str(i) for i in range(5,40,5)])
ax.set_title('Time Spent in Pit Lane (Relative)')
ax.axvline(1.0, color='white', linestyle='--', linewidth=1)
ax.grid(axis='x', linestyle='--')

plt.show()

In [ ]:
fig.savefig('pitstop_plot.png',dpi=300, bbox_inches='tight')